In [1]:
"""
Process Name : STEP 5 : CREATION OF MODEL
"""
#=======================================================================================================
### Required Imports ###
#============================================================================================================
try:
    import sys  # System-specific parameters and functions
    import logging # for logs
    from datetime import datetime  # Date and time
    import socket # network communication
    import nbformat
    from nbconvert.preprocessors import ExecutePreprocessor
    import os
    import pandas as pd
except Exception as err:
    print("Exception raised while importing the packages")
    print(f'Exception: {err}')
    #input("press Enter to Close")
    sys.exit()

In [2]:
#=========================================================================
### Initialization ###
#=========================================================================
try:
    path = os.getcwd()
    curr_time = datetime.now()

    ### Log Files declaration ###
    log_folder = os.path.join(path, 'Logs')
    log_date_fmt = str(curr_time.strftime('%Y')) + '-' + str(curr_time.strftime('%m')) + '-' + str(curr_time.strftime('%d')) + "_" + str(curr_time.strftime("%H")) + "-" + str(curr_time.strftime("%M"))

    audit_log_file = "Audit_Main.log"
    audit_log_file = os.path.join(log_folder, audit_log_file)
    
    error_log_file = "Error_Main.log"
    error_log_file = os.path.join(log_folder, error_log_file)

    ### Creating log folder ###
    if not os.path.exists(log_folder):
        os.makedirs(log_folder)
        
    ### Function: Logger setup ###
    def setup_logger(logger_name, log_file, level=logging.INFO):
        logger = logging.getLogger(logger_name)
        formatter = logging.Formatter(socket.gethostname()+' : '+'%(asctime)s : %(levelname)s : [%(filename)s:%(lineno)d] : %(message)s')

        fileHandler = logging.FileHandler(log_file, mode='w')

        fileHandler.setFormatter(formatter)

        streamHandler = logging.StreamHandler(sys.stdout)
        streamHandler.setFormatter(formatter)

        logger.setLevel(level)
        logger.addHandler(fileHandler)
        logger.addHandler(streamHandler)
        return logger
        
    ### Setting up the logger ###
    setup_logger('audit', audit_log_file, level=logging.INFO)
    setup_logger('error', error_log_file, level=logging.ERROR)

    audit_logger = logging.getLogger('audit')
    error_logger = logging.getLogger('error')
    audit_logger.info('Process start')

except Exception as err:
    print('Setting up the logger failed')
    print(f'Exception: {err}')
    #input("press Enter to Close")
    sys.exit()

192.168.1.112 : 2024-09-05 16:22:19,145 : INFO : [2204483403.py:45] : Process start


In [3]:
audit_logger.info('Saving functions')
def execute_notebook(jupyter_nb):
    with open(jupyter_nb, 'r', encoding='utf-8') as file:
        notebook = nbformat.read(file, as_version=4)
    
    ep = ExecutePreprocessor(timeout=1600, kernel_name='python3', allow_errors=False)  # allow_errors=False to halt on error

    try:
        ep.preprocess(notebook, {'metadata': {'path': os.path.dirname(jupyter_nb)}}) #run notebook
        
        with open(jupyter_nb, 'wt', encoding='utf-8') as file: ## Save the executed notebook only if successful to this point
            nbformat.write(notebook, file)
        print(f"Run successful for {jupyter_nb}")
        return True  # Return True if everything executed correctly without errors
    except Exception as e:
        # Catch any type of exception and stop the execution while reporting the error
        error_logger.info(f"Failure while running {jupyter_nb}: {e}")
        return False  # Return False indicating the execution did not complete successfully

192.168.1.112 : 2024-09-05 16:22:19,149 : INFO : [1532334409.py:1] : Saving functions


In [4]:
#==================================================================================================
### Loading main file ### 
#==================================================================================================
try:
    audit_logger.info('Loading main file')
    df = pd.read_excel("Main.xlsx")
except Exception as err:
    audit_logger.info('Loading main file - Failed')
    error_logger.error('Loading main file - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit() 

192.168.1.112 : 2024-09-05 16:22:19,153 : INFO : [1518559910.py:5] : Loading main file


In [5]:
#==================================================================================================
### Loading main file ### 
#==================================================================================================
try:
    audit_logger.info('Running notebooks')# Execute the notebooks if column "Run" reads 1
    for i in range(0, len(df)):
        if df.loc[i, "Run"] == 1:
            audit_logger.info(f'Running {df.loc[0, "File_name"]}')
            if not execute_notebook(df.loc[i, "File_name"]):
                audit_logger.info(f'Stopping further execution due to error in {df.loc[0, "File_name"]}')
                break  # Exit loop if any code fails
except Exception as err:
    audit_logger.info('Loading main file - Failed')
    error_logger.error('Loading main file - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit()

192.168.1.112 : 2024-09-05 16:22:19,273 : INFO : [2405010374.py:5] : Running notebooks
192.168.1.112 : 2024-09-05 16:22:19,276 : INFO : [2405010374.py:8] : Running pre_processing.ipynb


2024-09-05 16:22:26.291519: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 16:22:32.709234: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-05 16:22:32.712103: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-09-05 16:22:33.670699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Run successful for Prototype_model.ipynb


In [6]:
audit_logger.info('Process run success')
total_time = datetime.now() - curr_time
audit_logger.info(f'Total time taken = {total_time}')

192.168.1.112 : 2024-09-05 16:22:34,697 : INFO : [2214019538.py:1] : Process run success
192.168.1.112 : 2024-09-05 16:22:34,698 : INFO : [2214019538.py:3] : Total time taken = 0:00:15.553240
